In [1]:
import pandas as pd
import numpy as np

#from  datenguidepy import Query, get_regions, get_statistics
from datenguidepy.query_builder import Query
from datenguidepy.query_helper import get_regions, get_statistics
pd.set_option('display.max_colwidth', 200)

In [1]:
class DataQueryRegion():
    def __init__(self,region,query_list,query_des_list,query_word):
        self.region = region;
        self.query_list = query_list
        self.query_des_list = query_des_list
        self.query_word = query_word
        
    #returns a dataframe with 'query_field','short' and 'long' descriptions
    def get_query_fields_with_query_word(self):
        #df_stat = get_statistics().query("long_description.str.contains({})".format(self.query_word), engine='python')
        df_stat = get_statistics().query("long_description.str.contains('Kind')", engine='python')
        df_stat.reset_index(inplace=True)
        return df_stat
    
    def get_info_of_a_query_region(self,query_field):
        #get_statistics().query('statistic.str.contains("KIND03")', engine = 'python')
        query = Query.region(self.region)
        # add a field (the statstic) to the query
        q_field = query.add_field(query_field)#KINNW1 #SOZHI1
        return q_field.get_info()

    #returns a list of non-null data frames obtained from a query word
    def get_non_null_data_of_query_filed_region(self):
        df_stat = self.get_query_fields_with_query_word()
        df_list = []
        for stat,des in zip(df_stat['statistic'].to_list(),df_stat['short_description'].to_list()):
            #query = Query.region("name='Hamburg'")
            query = Query.region(self.region)
            # add a field (the statstic) to the query
            q_field = query.add_field(stat)
            #q_field.add_args({'GES': 'GESW'})
    
            #get info only for non-null entries
            if len(query.results().index)!=0:
                #print('Now we are looking at data of',stat,des)
        
                # we are only interested in the year and number
                df = query.results()[['year',stat]].sort_values('year')
                df.drop_duplicates(inplace=True,ignore_index=True)
                df.rename(columns={stat:des},inplace=True)
                #print(df)
                df_list.append(df)
        return df_list
    #return max, min year from a dataframe
    def _get_max_min_year_(self,df):
        return np.max(df['year']),np.min(df['year'])

    #returns a dataframe froma list of query_fileds
    def get_data_combined_query_list_region(self):
        #get max & min year avaiable from all data sources with a query word
        #df_list = self.get_non_null_data_of_query_filed_region()
        #maxyear, minyear = self._get_max_min_year_(pd.concat(df_list))
                
        #or manually set max, min year 
        maxyear, minyear = int(2019), int(1994)
        
        yeardata = {'year': list(np.arange(minyear,maxyear))}
        
        #create a dataframe with a column year
        df = pd.DataFrame(yeardata,columns=['year'])
        for stat,des in zip(self.query_list,self.query_des_list):
            #query = Query.region("name='Hamburg'")
            query = Query.region(self.region)
            # add a field (the statstic) to the query
            q_field = query.add_field(stat)
    
            # we are only interested in the year and number
            dftmp = query.results()[['year',stat]].sort_values('year')
            dftmp.drop_duplicates(inplace=True,ignore_index=True)
            dftmp.rename(columns={stat:des},inplace=True)
            #print(dftmp)
            df = pd.merge(df,dftmp,how='left',on='year')
        return df